In [1]:
from sage.all import *
from sage.crypto.sbox import *
from sage.crypto.sboxes import sboxes
import numpy as np
import pickle
import pandas as pd

In [2]:
# Modified from https://github.com/abrari/block-cipher-testing

# Works only on square s-boxes
def sbox_bic(sbox):

    maxCorr = 0
    m = len(sbox)
    for i in range(m):
        ei = 2**i 
        for j in range(m):
            for k in range(m):
                if j != k:
                    avalanche_vec_j = np.zeros(2**m, dtype=float)
                    avalanche_vec_k = np.zeros(2**m, dtype=float)

                    # Each possible input
                    for X in range(2**m):
                        ej = 2**j 
                        ek = 2**k 

                        dei = sbox[X] ^ sbox[X ^ ei]
                        dej = (dei & ej) >> j
                        dek = (dei & ek) >> k 

                        avalanche_vec_j[X] = dej 
                        avalanche_vec_k[X] = dek
                    corr = abs(np.corrcoef(avalanche_vec_j, avalanche_vec_k)[0,1])
                
                    if maxCorr < corr:
                        maxCorr = corr
    return maxCorr

sbox_bic(sboxes["AES"])


0.03225806451612903
0.06575124259478841
0.06696391397229075
0.034347664042053645
0.030303030303030304
0.059863071616150634
0.03126526997403612
0.03225806451612903
0.002944085489318884
0.003939053763075926
0.059863071616150634
0.0009775171065493646
0.06575124259478841
0.06253053994807224
0.0657512425947884
0.002944085489318884
0.019772733298509516
0.040394088669950735
0.002944085489318884
0.040394088669950735
0.03138824102871722
0.06696391397229073
0.003939053763075927
0.019772733298509516
0.08304547985373997
0.059085806446138894
0.04350001325672093
0.12598815766974242
0.03434766404205365
0.059863071616150634
0.040394088669950735
0.08304547985373997
0.059863071616150634
0.07192118226600985
0.03138824102871722
0.030303030303030304
0.0009775171065493646
0.002944085489318884
0.059085806446138894
0.059863071616150634
0.002944085489318884
0.03126526997403612
0.059863071616150634
0.0657512425947884
0.040394088669950735
0.04350001325672093
0.07192118226600985
0.002944085489318884
0.06277648205

0.1341246360271594

In [3]:
# Modified version to get the average SAC of an sbox from:
#       https://github.com/abrari/block-cipher-testing
def avg_sac(sbox):
    m = len(sbox)
    sac_mat = np.array([np.zeros(m, dtype=float) for _ in range(m)])

    for i in range(m):
        ei = 2**i 
        for j in range(m):
            ej = 2**j 
            for X in range(2**m):
                dei = sbox[X] ^ sbox[X ^ ei]
                sac_mat[i][j] += (dei & ej) >> j 
    
    output_len = 2**m
    for i in range(m):
        for j in range(m):
            sac_mat[i][j] /= output_len

    sum = 0 
    for i in range(m):
        sum += np.sum(sac_mat[i])
    avg = sum / (m**2)
    return avg 

avg_sac(sboxes["AES"])

0.5048828125

In [4]:
nl_dict = {}
lp_dict = {}
dp_dict = {}

for sbox in sboxes.values():
    if len(sbox) == 8:
        nl = sbox.nonlinearity()
        lp = sbox.maximal_linear_bias_relative()
        dp = sbox.maximal_difference_probability()
        if nl in nl_dict.keys():
            nl_dict[nl]+=1
        else:
            nl_dict[nl] = 1

        if lp in lp_dict.keys():
            lp_dict[lp] += 1
        else:
            lp_dict[lp] = 1

        if dp in dp_dict.keys():
            dp_dict[dp] += 1
        else:
            dp_dict[dp] = 1

print(nl_dict)
print(lp_dict)
print(dp_dict)

{112: 11, 94: 5, 102: 1, 100: 8, 96: 20, 0: 1, 88: 1, 64: 4, 104: 4, 90: 1, 82: 1, 92: 1}
{0.0625: 11, 0.1328125: 5, 0.1015625: 1, 0.109375: 8, 0.125: 20, 0.5: 1, 0.15625: 1, 0.25: 4, 0.09375: 4, 0.1484375: 1, 0.1796875: 1, 0.140625: 1}
{0.015625: 12, 0.03125: 17, 0.0390625: 11, 0.046875: 6, 0.5: 2, 0.0625: 6, 0.25: 4}


In [5]:
print(len(nl_dict))
print(len(lp_dict))
print(len(dp_dict))

12
12
7


In [6]:
sboxes["GIFT"]

(1, 10, 4, 12, 6, 15, 3, 9, 2, 13, 11, 7, 5, 0, 8, 14)

In [7]:
def sbox_to_arr(sbox):
    return [num for num in sbox]

Pickle the info for 8-bit sbox

In [8]:
pickle_filename = "sboxes_info.pkl"
df_dict = {}
for name, sbox in sboxes.items():
    if len(sbox) == 8:
        print("pickling:", name)
        row = {}
        row["box"] = [num for num in sbox] 
        row["inverse"] = [num for num in sbox.inverse()] if sbox.is_permutation() else []
        row["nonlinearity"] = int(sbox.nonlinearity())
        row["linear_probability"] = float(sbox.maximal_linear_bias_relative())
        row["differential_probability"] = float(sbox.maximal_difference_probability())
        row["boomerang_uniformity"] = int(sbox.boomerang_uniformity()) if sbox.is_permutation() else -1
        row["diff_branch"] = int(sbox.differential_branch_number())
        row["linear_branch"] = int(sbox.linear_branch_number())
        row["linearity"] = int(sbox.linearity())
        row["bic"] = sbox_bic(sbox) 
        row["sac"] = avg_sac(sbox) 

        df_dict[name] = row
df = pd.DataFrame.from_dict(df_dict).T
with open(pickle_filename, "wb") as f:     
    pickle.dump(df, f)

AES
0.03225806451612903
0.06575124259478841
0.06696391397229075
0.034347664042053645
0.030303030303030304
0.059863071616150634
0.03126526997403612
0.03225806451612903
0.002944085489318884
0.003939053763075926
0.059863071616150634
0.0009775171065493646
0.06575124259478841
0.06253053994807224
0.0657512425947884
0.002944085489318884
0.019772733298509516
0.040394088669950735
0.002944085489318884
0.040394088669950735
0.03138824102871722
0.06696391397229073
0.003939053763075927
0.019772733298509516
0.08304547985373997
0.059085806446138894
0.04350001325672093
0.12598815766974242
0.03434766404205365
0.059863071616150634
0.040394088669950735
0.08304547985373997
0.059863071616150634
0.07192118226600985
0.03138824102871722
0.030303030303030304
0.0009775171065493646
0.002944085489318884
0.059085806446138894
0.059863071616150634
0.002944085489318884
0.03126526997403612
0.059863071616150634
0.0657512425947884
0.040394088669950735
0.04350001325672093
0.07192118226600985
0.002944085489318884
0.0627764

/home/aolguin1/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/aolguin1/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


nan
0.5773502691896257
0.5773502691896257
0.9999999999999998
nan
0.5773502691896257
0.5773502691896257
nan
nan
nan
nan
nan
nan
nan
0.5773502691896257
nan
1.0
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.5773502691896257
nan
1.0
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.9999999999999998
nan
0.5773502691896257
0.5773502691896257
nan
0.5773502691896257
0.5773502691896257
nan
nan
nan
nan
nan
nan
nan
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.5773502691896257
nan
0.33333333333333337
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.5773502691896257
nan
0.33333333333333337
nan
0.5773502691896257
0.5773502691896257
0.9999999999999998
nan
0.5773502691896257
0.5773502691896257
nan
nan
nan
nan
nan
nan
nan
0.5773502691896257
nan
1.0
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.5773502691896257
nan
1.0
0.5773502691896257
nan
0.33333333333333337
0.33333333333333337
0.9999999999999998
nan
0.5773502691

In [9]:
df

,box,inverse,nonlinearity,linear_probability,differential_probability,boomerang_uniformity,diff_branch,linear_branch,linearity,bic,sac
AES,"[99, 124, 119, 123, 242, 107, 111, 197, 48, 1,...","[82, 9, 106, 213, 48, 54, 165, 56, 191, 64, 16...",112,0.0625,0.015625,6,2,2,32,0.134125,0.504883
ARIA_s2,"[226, 78, 84, 252, 148, 194, 74, 204, 98, 13, ...","[48, 104, 153, 27, 135, 185, 33, 120, 80, 57, ...",112,0.0625,0.015625,6,2,2,32,0.134125,0.50293
Anubis,"[167, 211, 230, 113, 208, 172, 77, 121, 58, 20...","[167, 211, 230, 113, 208, 172, 77, 121, 58, 20...",94,0.132812,0.03125,18,2,2,68,0.254902,0.500244
BelT,"[177, 148, 186, 200, 10, 8, 245, 59, 54, 109, ...","[10, 167, 28, 156, 17, 58, 144, 145, 5, 206, 4...",102,0.101562,0.03125,20,2,2,52,0.168409,0.499512
CLEFIA_S0,"[87, 73, 209, 198, 47, 51, 116, 251, 149, 109,...","[69, 187, 253, 144, 163, 135, 40, 226, 106, 94...",100,0.109375,0.039062,32,3,3,56,0.333333,0.539062
CLEFIA_S1,"[108, 218, 195, 233, 78, 157, 10, 61, 184, 54,...","[201, 232, 231, 249, 129, 166, 146, 25, 153, 1...",112,0.0625,0.015625,6,2,2,32,0.131696,0.495605
CMEA,"[217, 35, 95, 230, 202, 104, 151, 176, 123, 24...",[],96,0.125,0.046875,-1,1,2,64,0.296591,0.516113
CSA,"[58, 234, 104, 254, 51, 233, 136, 26, 131, 207...","[159, 197, 190, 51, 59, 175, 25, 230, 46, 195,...",94,0.132812,0.046875,18,2,2,68,0.292611,0.491455
CSS,"[51, 115, 59, 38, 99, 35, 107, 118, 62, 126, 5...","[51, 59, 115, 21, 83, 91, 19, 117, 61, 53, 125...",0,0.5,0.5,256,2,2,256,1.0,0.28125
CS_cipher,"[41, 13, 97, 64, 156, 235, 158, 143, 31, 133, ...","[41, 13, 97, 64, 156, 235, 158, 143, 31, 133, ...",96,0.125,0.0625,256,2,2,64,1.0,0.435059


In [10]:
type(df.T.to_dict()["AES"]["nonlinearity"])

int